In [0]:
# Biblioteca para Big Data
import pyspark
import os
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

# Datos masivos - Tarea 1
__Andrés Urbano Guillermo Gerardo__


__Objetivo__: explorar la carga, uso y manipulación de RDDs y DataFrames en PySpark.

__Intrucciones__:
1. Usando databricks, debes crear una libreta y cargar una base de datos (escoge la que prefieras de "FIFA 20 complete player dataset"). Al final encontrarás el vínculo para la descarga.
2. Debes cargar los datos a databricks y cargarlos a tu libreta como RDD y como dataFrame.
3. Una vez que cargues los datos, debes realizar dos operaciones usando el RDD y dos operaciones usando el dataFrame.
4. También debes generar una consulta en SQL

In [0]:
spark = SparkSession.builder.appName('tarea1_dm').getOrCreate()

## Operaciones con RDDs

Cargamos nuestros conjunto de datos:

In [0]:
mainpath = '/FileStore/tables'
filename = 'players_18.csv'
fullpath = os.path.join(mainpath, filename)
raw_data = spark.sparkContext.textFile(fullpath)

In [0]:
# Observamos las primeras dos columnas
# taken: devuelve una lista de cadenas que sera cada observación
# for row in raw_data.take(2):
#     print(row)

In [0]:
# Obtenemos los nombres de las columnas
features_names = raw_data.take(1)[0]
# Obtenemos sólo los datos
data = raw_data.filter(lambda row: row != features_names)
# data.take(1)

## Operaciones de transformación RDD
Extraemos las caracteristicas que nos interesan:

In [0]:
def split_fields(row):    
    columns_names = row.split(',')        
    name =  columns_names[2]
    age =  int(columns_names[4])
    height = int(columns_names[6])
    weight = int(columns_names[7])
    nationality = columns_names[8]
    club = columns_names[9]
    rank = int(columns_names[10])
    return (name, age, height, weight, nationality, club, rank)

### Map

In [0]:
rdd_fifa18 = data.map(split_fields)

for player in rdd_fifa18.take(2):
    print(player)

('Cristiano Ronaldo', 32, 185, 80, 'Portugal', 'Real Madrid', 94)
('L. Messi', 30, 170, 72, 'Argentina', 'FC Barcelona', 93)


__Ejemplo 1: ¿Cuántos jugadores existen por país?__

Para resolver la pregunta podemos utilizar diccionarios de Python, ya que las llaves son únicas, podemos extrapolar la misma idea con el paradigma MapReduce con los RDD

In [0]:
countries = {}
country_list = ['Portugal', 'Argentina', 'Brazil', 'Germany', 'Uruguay', 'Poland', 'Spain', 'Belgium', 'Spain', 'Argentina', 'Germany', 'Belgium', 'Belgium', 'Italy', 'Italy', 'Argentina', 'Wales', 'Croatia', 'Chile', 'Slovenia', 'Argentina', 'France', 'Spain']
for country in country_list:
    if country not in countries:
        countries[country] = 1
    countries[country] += 1
print(countries)

{'Portugal': 2, 'Argentina': 5, 'Brazil': 2, 'Germany': 3, 'Uruguay': 2, 'Poland': 2, 'Spain': 4, 'Belgium': 4, 'Italy': 3, 'Wales': 2, 'Croatia': 2, 'Chile': 2, 'Slovenia': 2, 'France': 2}


In [0]:
# Obtenemos todas las nacionalidades y iniciamos la cuenta en 1
rdd_nationalities = rdd_fifa18.map(lambda row: (row[4], 1))
for nationality  in rdd_nationalities.take(20):
    print(nationality)

('Portugal', 1)
('Argentina', 1)
('Brazil', 1)
('Germany', 1)
('Uruguay', 1)
('Poland', 1)
('Spain', 1)
('Belgium', 1)
('Spain', 1)
('Argentina', 1)
('Germany', 1)
('Belgium', 1)
('Belgium', 1)
('Italy', 1)
('Italy', 1)
('Argentina', 1)
('Wales', 1)
('Croatia', 1)
('Chile', 1)
('Slovenia', 1)


### ReduceByKey

In [0]:
rdd_nationalities2 = rdd_nationalities.reduceByKey(lambda v1, v2: v1 + v2)
results = rdd_nationalities2.collect()
results

Out[55]: [('Portugal', 350),
 ('Argentina', 964),
 ('Brazil', 799),
 ('Uruguay', 147),
 ('Poland', 320),
 ('Belgium', 274),
 ('Slovenia', 64),
 ('France', 958),
 ('Netherlands', 422),
 ('Gabon', 12),
 ('Denmark', 342),
 ('Slovakia', 64),
 ('Austria', 267),
 ('Greece', 97),
 ('Bosnia Herzegovina', 56),
 ('Costa Rica', 28),
 ('Switzerland', 232),
 ('Algeria', 56),
 ('Ecuador', 37),
 ('Montenegro', 25),
 ('Kenya', 7),
 ('Korea Republic', 331),
 ('Norway', 285),
 ('Mexico', 360),
 ('Iceland', 46),
 ('Finland', 60),
 ('Nigeria', 121),
 ('Republic of Ireland', 399),
 ('Ukraine', 48),
 ('United States', 368),
 ('Uzbekistan', 5),
 ('Ghana', 115),
 ('Venezuela', 49),
 ('Albania', 35),
 ('Paraguay', 69),
 ('Cape Verde', 22),
 ('Northern Ireland', 85),
 ('Tunisia', 30),
 ('Togo', 9),
 ('Estonia', 8),
 ('Peru', 28),
 ('Hungary', 38),
 ('Iran', 17),
 ('Angola', 15),
 ('Australia', 196),
 ('Scotland', 300),
 ('Mozambique', 3),
 ('South Africa', 77),
 ('Saudi Arabia', 331),
 ('Kosovo', 31),
 ('Hondur

### SortBy

In [0]:
# Ordenar jugadores
rdd_nationalities2.sortBy(lambda row: row[1], ascending=False).take(5)

Out[56]: [('England', 1608),
 ('Germany', 1141),
 ('Spain', 1002),
 ('Argentina', 964),
 ('France', 958)]

## Operaciones con Dataframes

In [0]:
df_fifa18 = spark.read.csv(mainpath, header=True, inferSchema=True)
# Seleccionamos algunas caracteristicas
df_fifa18 = df_fifa18.select('short_name', 'age', 'height_cm', 'weight_kg',
                            'nationality', 'club', 'value_eur', 'preferred_foot')
df_fifa18.show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|Cristiano Ronaldo| 32|      185|       80|   Portugal|        Real Madrid| 95500000|         Right|
|         L. Messi| 30|      170|       72|  Argentina|       FC Barcelona|105000000|          Left|
|           Neymar| 25|      175|       68|     Brazil|Paris Saint-Germain|123000000|         Right|
|         M. Neuer| 31|      193|       92|    Germany|  FC Bayern München| 61000000|         Right|
|        L. Suárez| 30|      182|       86|    Uruguay|       FC Barcelona| 97000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



In [0]:
# Recortar el dataframe
df_small = df_fifa18.limit(10)
# Convertirlo en un Dataframe de Pandas (Solo si suficientemente pequeño)
df_pandas = df_small.toPandas()
df_pandas.tail()

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
5,R. Lewandowski,28,185,79,Poland,FC Bayern München,92000000,Right
6,De Gea,26,193,76,Spain,Manchester United,64500000,Right
7,E. Hazard,26,173,76,Belgium,Chelsea,90500000,Right
8,Sergio Ramos,31,183,75,Spain,Real Madrid,52000000,Right
9,G. Higuaín,29,184,87,Argentina,Juventus,77000000,Right


In [0]:
# Conocer el esquema
df_fifa18.printSchema()

root
 |-- short_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- nationality: string (nullable = true)
 |-- club: string (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- preferred_foot: string (nullable = true)



### Drop and Rename Columns

In [0]:
# Elimnar columnas
df_fifa18 = df_fifa18.drop('preferred_foot')
# Renombar columnas
df_fifa18 = df_fifa18.withColumnRenamed('value_eur', 'euros')

In [0]:
df_fifa18.show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+
|       short_name|age|height_cm|weight_kg|nationality|               club|    euros|
+-----------------+---+---------+---------+-----------+-------------------+---------+
|Cristiano Ronaldo| 32|      185|       80|   Portugal|        Real Madrid| 95500000|
|         L. Messi| 30|      170|       72|  Argentina|       FC Barcelona|105000000|
|           Neymar| 25|      175|       68|     Brazil|Paris Saint-Germain|123000000|
|         M. Neuer| 31|      193|       92|    Germany|  FC Bayern München| 61000000|
|        L. Suárez| 30|      182|       86|    Uruguay|       FC Barcelona| 97000000|
+-----------------+---+---------+---------+-----------+-------------------+---------+
only showing top 5 rows



In [0]:
# Ordenar columnas
df_fifa18.orderBy('euros', ascending=False).show(5)

+----------+---+---------+---------+-----------+-------------------+---------+
|short_name|age|height_cm|weight_kg|nationality|               club|    euros|
+----------+---+---------+---------+-----------+-------------------+---------+
|    Neymar| 25|      175|       68|     Brazil|Paris Saint-Germain|123000000|
| Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|
|  L. Messi| 30|      170|       72|  Argentina|       FC Barcelona|105000000|
| L. Suárez| 30|      182|       86|    Uruguay|       FC Barcelona| 97000000|
|  L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|
+----------+---+---------+---------+-----------+-------------------+---------+
only showing top 5 rows



### Filtrar datos

In [0]:
df_fifa18.where(df_fifa18.age > 35).show(5)

+-----------+---+---------+---------+-----------+--------------------+--------+
| short_name|age|height_cm|weight_kg|nationality|                club|   euros|
+-----------+---+---------+---------+-----------+--------------------+--------+
|  G. Buffon| 39|      191|       94|      Italy|            Juventus| 4500000|
|A. Barzagli| 36|      187|       87|      Italy|            Juventus| 5000000|
|     Aduriz| 36|      182|       78|      Spain|Athletic Club de ...|11000000|
|   Casillas| 36|      185|       84|      Spain|            FC Porto| 3500000|
|   S. Eto'o| 36|      179|       75|   Cameroon|         Antalyaspor| 6500000|
+-----------+---+---------+---------+-----------+--------------------+--------+
only showing top 5 rows



In [0]:
df_fifa18.where((df_fifa18.age > 15) & 
                (df_fifa18.club == 'Juventus')).show(5)

+------------+---+---------+---------+-----------+--------+--------+
|  short_name|age|height_cm|weight_kg|nationality|    club|   euros|
+------------+---+---------+---------+-----------+--------+--------+
|  G. Higuaín| 29|      184|       87|  Argentina|Juventus|77000000|
|   G. Buffon| 39|      191|       94|      Italy|Juventus| 4500000|
|G. Chiellini| 32|      187|       85|      Italy|Juventus|38000000|
|   P. Dybala| 23|      177|       73|  Argentina|Juventus|79000000|
| Alex Sandro| 26|      181|       81|     Brazil|Juventus|44000000|
+------------+---+---------+---------+-----------+--------+--------+
only showing top 5 rows



### Funciones de agregación

In [0]:
df_fifa18.groupBy('club') \
    .sum('euros') \
    .orderBy('sum(euros)', ascending=False) \
    .show(5)

+-----------------+----------+
|             club|sum(euros)|
+-----------------+----------+
|      Real Madrid|1724600000|
|     FC Barcelona|1584300000|
|FC Bayern München|1437600000|
|  Manchester City|1424460000|
|         Juventus|1352575000|
+-----------------+----------+
only showing top 5 rows



En esta consulta podemos encontrar los 5 equipos más caros.

In [0]:
df_fifa18.groupBy('club') \
    .agg(sum('euros').alias('ValorTotal'), mean('age').alias('edad promedio')) \
    .orderBy('ValorTotal', ascending=False) \
    .show(5)

+-----------------+----------+------------------+
|             club|ValorTotal|     edad promedio|
+-----------------+----------+------------------+
|      Real Madrid|1724600000| 24.76271186440678|
|     FC Barcelona|1584300000|24.982456140350877|
|FC Bayern München|1437600000|25.102040816326532|
|  Manchester City|1424460000| 23.90909090909091|
|         Juventus|1352575000|27.338983050847457|
+-----------------+----------+------------------+
only showing top 5 rows



En esta consulta podemos encontrar varias funciones de agreagación que nos indica el valor todal del club como también el valor promedio de los jugadores.

In [0]:
# Crear columnas
df_fifa18.withColumn('pesos', df_fifa18.euros * 18).show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+-----------+
|       short_name|age|height_cm|weight_kg|nationality|               club|    euros|      pesos|
+-----------------+---+---------+---------+-----------+-------------------+---------+-----------+
|Cristiano Ronaldo| 32|      185|       80|   Portugal|        Real Madrid| 95500000| 1719000000|
|         L. Messi| 30|      170|       72|  Argentina|       FC Barcelona|105000000| 1890000000|
|           Neymar| 25|      175|       68|     Brazil|Paris Saint-Germain|123000000|-2080967296|
|         M. Neuer| 31|      193|       92|    Germany|  FC Bayern München| 61000000| 1098000000|
|        L. Suárez| 30|      182|       86|    Uruguay|       FC Barcelona| 97000000| 1746000000|
+-----------------+---+---------+---------+-----------+-------------------+---------+-----------+
only showing top 5 rows



## Consultas en SQL

In [0]:
# Guarada el dataframe como una tabla de SQL
df_fifa18.registerTempTable('FIFA18')

In [0]:
consulta = '''
    SELECT club, SUM(euros) AS ValorTotal
    FROM FIFA18
    GROUP BY club
    ORDER BY ValorTotal DESC
'''
spark.sql(consulta).show(5)

+-----------------+----------+
|             club|ValorTotal|
+-----------------+----------+
|      Real Madrid|1724600000|
|     FC Barcelona|1584300000|
|FC Bayern München|1437600000|
|  Manchester City|1424460000|
|         Juventus|1352575000|
+-----------------+----------+
only showing top 5 rows



In [0]:
%sql
SELECT club, SUM(euros) AS valor_total
FROM FIFA18
GROUP BY club
ORDER BY valor_total DESC
limit 5

club,valor_total
Real Madrid,1724600000
FC Barcelona,1584300000
FC Bayern München,1437600000
Manchester City,1424460000
Juventus,1352575000


In [0]:
%sql
SELECT short_name, age, club FROM FIFA18
WHERE age > 25 AND club='Juventus'
LIMIT 5

short_name,age,club
G. Higuaín,29,Juventus
G. Buffon,39,Juventus
G. Chiellini,32,Juventus
Alex Sandro,26,Juventus
M. Pjanić,27,Juventus


## Resources
- https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset?select=players_18.csv
- https://github.com/blancavazquez/CursoDatosMasivosI/blob/master/notebooks/1a_hello_pyspark.ipynb